In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
url="https://turbo.az/"
response=requests.get(url)
soup=BeautifulSoup(response.text,'html')

In [3]:
essNames = []
essYear = []
essEngine = []
essRide = []
essPrice = []
essChange = []
essLoan = []
essNew = []
essLoc = []

In [4]:
links1 = soup.find_all('div',class_='products-i vipped salon')
links2 = soup.find_all('div',class_='products-i vipped featured salon')
links3 = soup.find_all('div',class_='products-i vipped')
links4 = soup.find_all('div',class_='products-i')
links = links1 + links2 + links3 + links4

In [5]:
for i in links:
    name = i.find(class_='products-name').text
    gen = i.find(class_='products-attributes').text
    year = gen[:4]
    engine = gen[7:12]
    ride = gen[12:]
    ride_new = re.subn(r'\s','',ride,1)[0]
    price = i.find(class_='products-price').text
    price = re.subn(r'\s','',price,1)[0]
    price = re.sub(r'\$', 'USD', price)
    price = re.sub(r'\€', 'EUR', price)
    city = i.find(class_='products-bottom').text
    city = re.search(r'\w+',city).group()
        
    if i.find(class_='products-barter'):
              essChange.append('Yes')
    else:
              essChange.append('No')
    
    if i.find(class_='products-loan'):
              essLoan.append('Yes')
    else:
              essLoan.append('No')
    
    if re.search(r'\d+',ride_new).group()=='0':
        essNew.append('Yes')
    else:
        essNew.append('No')
    
    url_cbar="https://www.cbar.az/currency/rates"
    response_cbar=requests.get(url_cbar)
    soup_cbar=BeautifulSoup(response_cbar.text,'html')
    val = soup_cbar.find_all('div',class_='table_row')
    usd = ''
    eur = ''
        
    for i in val:
            if 'usd' in i.text:
                usd = i.text
            elif 'eur' in i.text:
                eur = i.text
    
    usd = float(re.search(r'\d+\.\d+',usd).group())
    eur = float(re.search(r'\d+\.\d+',eur).group())
    
    if 'USD' in price:
        num = re.sub(r'\s','',price)
        num = int(re.search(r'\d+',num).group())
        price = '{} AZN'.format(int(num*usd))
    
    if 'EUR' in price:
        num = re.sub(r'\s','',price)
        num = int(re.search(r'\d+',num).group())
        price = '{} AZN'.format(int(num*eur))

    
    essNames.append(name)    
    essYear.append(year)
    essEngine.append(engine)
    essRide.append(ride)
    essPrice.append(price)
    essLoc.append(city)
    
    a = []
    b = []
    c = []
    d = []
    e = []
    f = []
    g = []
    h = []
    l = []


In [6]:
for i in zip(essNames,essYear,essEngine,essRide,essPrice,essChange,essLoan,essNew,essLoc):
    a.append(i[0])
    b.append(i[1])
    c.append(i[2])
    d.append(i[3])
    e.append(i[4])
    f.append(i[5])
    g.append(i[6])
    h.append(i[7])
    l.append(i[8])

In [7]:
summary= {"Name" : a, "Production Year" : b, 
          "Engine Size" : c, 'New' : h, "Ride" : d, 'Price' : e, 
          'City': l, 'Change' : f, 'Loan' : g}

turbo=pd.DataFrame(summary)

In [8]:
turbo.head(10)

,Name,Production Year,Engine Size,New,Ride,Price,City,Change,Loan
0,Changan CS 75,2021,2.0 L,Yes,0 km,63900 AZN,Bakı,Yes,No
1,Mercedes E 220,2012,2.2 L,No,186 000 km,36550 AZN,Bakı,Yes,No
2,Honda Civic,2016,1.5 L,No,64 225 km,28730 AZN,Bakı,No,No
3,LADA (VAZ) Niva,1995,1.7 L,No,124 500 km,10700 AZN,Sumqayıt,No,Yes
4,Haval Jolion,2021,1.5 L,Yes,0 km,40900 AZN,Bakı,Yes,No
5,Alfa Romeo Stelvio,2020,2.0 L,Yes,0 km,99705 AZN,Bakı,No,No
6,Khazar SD,2021,1.7 L,Yes,0 km,16300 AZN,Bakı,No,No
7,Audi A6,2021,2.0 L,Yes,0 km,117231 AZN,Bakı,Yes,No
8,Mercedes S 500,2012,4.7 L,No,95 000 km,62050 AZN,Bakı,Yes,No
9,Land Rover Range Rover,2012,5.0 L,No,145 000 km,62050 AZN,Bakı,No,No


In [9]:
turbo.to_csv('turboAzScrapping.csv',index=True,index_label = 'ID')